**Spark SQL**

### SQL in Spark: Overview and Usage

Spark SQL is a module in Apache Spark that provides a structured data processing interface using SQL-like queries. It bridges the gap between the relational and procedural paradigms, allowing data analysts and developers to interact with data using SQL while leveraging Spark's scalability and performance.

**Key Features of Spark SQL:**
- **Unified Data Access**: Access data from various sources like JSON, Parquet, Avro, Hive, and JDBC.
- **Schema-Based Processing**: Enables schema definition and enforcement for structured data.
- **Integration with DataFrames**: Allows seamless transitions between SQL queries and DataFrame APIs.
- **Optimized Query Execution**: Uses the Catalyst Optimizer for efficient query planning.

*Example 1: Running SQL queries on a DataFrame*

In [0]:
# Load data from JSON
flight_data_json = "dbfs:/mnt/data/data/flight-data/json/2015-summary.json"
df = spark.read.format("json").load(flight_data_json)

# Register DataFrame as a SQL table
df.createOrReplaceTempView("flight_data")

# Query using Spark SQL
query = """
SELECT DEST_COUNTRY_NAME, SUM(count) AS total_flights 
FROM flight_data 
GROUP BY DEST_COUNTRY_NAME 
ORDER BY total_flights DESC 
LIMIT 10
"""
sql_result = spark.sql(query)
sql_result.show()

+------------------+-------------+
| DEST_COUNTRY_NAME|total_flights|
+------------------+-------------+
|     United States|     411337.0|
|            Canada|       8399.0|
|            Mexico|       7140.0|
|    United Kingdom|       2025.0|
|             Japan|       1548.0|
|           Germany|       1468.0|
|Dominican Republic|       1353.0|
|       South Korea|       1048.0|
|       The Bahamas|        955.0|
|            France|        935.0|
+------------------+-------------+



---

### Creating and Managing Tables with Spark SQL

Spark SQL allows users to create and manage tables, supporting both temporary and permanent tables.

**1. Temporary Views:**
Temporary views exist only within the Spark session and are not persistent.

*Example 2: Creating and querying a temporary view*

In [0]:
# Creating a temporary view
df.createOrReplaceTempView("temp_flights")

# Querying the temporary view
query = """
SELECT ORIGIN_COUNTRY_NAME, COUNT(*) AS num_flights
FROM temp_flights
GROUP BY ORIGIN_COUNTRY_NAME
HAVING num_flights > 10
ORDER BY num_flights DESC
LIMIT 5
"""
spark.sql(query).show()

# More complex query with multiple aggregations
query_advanced = """
SELECT ORIGIN_COUNTRY_NAME, SUM(count) AS total_passengers, AVG(count) AS avg_flights
FROM temp_flights
GROUP BY ORIGIN_COUNTRY_NAME
ORDER BY total_passengers DESC
LIMIT 10
"""
spark.sql(query_advanced).show()

+-------------------+-----------+
|ORIGIN_COUNTRY_NAME|num_flights|
+-------------------+-----------+
|      United States|        132|
+-------------------+-----------+

+-------------------+----------------+------------------+
|ORIGIN_COUNTRY_NAME|total_passengers|       avg_flights|
+-------------------+----------------+------------------+
|      United States|        411966.0|3120.9545454545455|
|             Canada|          8483.0|            8483.0|
|             Mexico|          7187.0|            7187.0|
|     United Kingdom|          1970.0|            1970.0|
|              Japan|          1496.0|            1496.0|
| Dominican Republic|          1420.0|            1420.0|
|            Germany|          1336.0|            1336.0|
|        The Bahamas|           986.0|             986.0|
|             France|           952.0|             952.0|
|              China|           920.0|             920.0|
+-------------------+----------------+------------------+



**2. Global Temporary Views:**
Global temporary views are accessible across multiple sessions using the `global_temp` database.

*Example 3: Creating and querying a global temporary view*

In [0]:
# Create a global temporary view
df.createOrReplaceGlobalTempView("global_flights")

# Query the global view
query = "SELECT * FROM global_temp.global_flights WHERE count > 500"
spark.sql(query).show()

+------------------+-------------------+------+
| DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+------------------+-------------------+------+
|        Costa Rica|      United States|   588|
|     United States|        Netherlands|   660|
|       The Bahamas|      United States|   955|
|       El Salvador|      United States|   561|
|            Mexico|      United States|  7140|
|     United States|         Costa Rica|   608|
|          Colombia|      United States|   873|
|     United States|            Jamaica|   712|
|            Panama|      United States|   510|
|     United States|        The Bahamas|   986|
|     United States|              China|   920|
|     United States| Dominican Republic|  1420|
|     United States|      United States|370002|
|           Germany|      United States|  1468|
|     United States|        South Korea|   827|
|     United States|        El Salvador|   508|
|            Canada|      United States|  8399|
|           Jamaica|      United States|

**3. Managed Tables:**
Managed tables are stored in Spark's warehouse directory and managed by Spark.

*Example 4: Creating a managed table and inserting data*

In [0]:
spark.sql("CREATE TABLE IF NOT EXISTS managed_table (DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)")
spark.sql("INSERT INTO managed_table SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, TRY_CAST(count AS LONG) AS count FROM temp_flights")
spark.sql("SELECT * FROM managed_table").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania| NULL|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

---

### Performance Optimization with Catalyst Optimizer

The Catalyst Optimizer is a key component of Spark SQL, designed to enhance query performance through logical and physical plan optimization.

**Key Techniques:**
- **Predicate Pushdown:** Filters are applied early to reduce data scanning.
- **Column Pruning:** Reads only required columns from the data source.
- **Join Optimization:** Reorders joins for optimal execution.
- **Query Caching:** Uses in-memory storage for frequently accessed data.

*Example 6: Using query caching for performance optimization*

In [0]:
# Cache query result
cached_result = spark.sql("""
SELECT DEST_COUNTRY_NAME, SUM(count) AS total_flights
FROM flight_data
GROUP BY DEST_COUNTRY_NAME
""")
cached_result.cache()
cached_result.count()
cached_result.show()

+--------------------+-------------+
|   DEST_COUNTRY_NAME|total_flights|
+--------------------+-------------+
|            Anguilla|         41.0|
|              Russia|        176.0|
|            Paraguay|         60.0|
|             Senegal|         40.0|
|              Sweden|        118.0|
|            Kiribati|         26.0|
|              Guyana|         64.0|
|         Philippines|        134.0|
|            Djibouti|          1.0|
|            Malaysia|          2.0|
|           Singapore|          3.0|
|                Fiji|         24.0|
|              Turkey|        138.0|
|                Iraq|          1.0|
|             Germany|       1468.0|
|              Jordan|         44.0|
|               Palau|         30.0|
|Turks and Caicos ...|        230.0|
|              France|        935.0|
|              Greece|         30.0|
+--------------------+-------------+
only showing top 20 rows



---

### Using Databricks SQL for Queries and Visualization

Databricks SQL provides a user-friendly interface for running SQL queries and creating visualizations.

**Steps to Use Databricks SQL:**
1. Navigate to the SQL editor in Databricks.
2. Select or create a SQL warehouse.
3. Write and execute SQL queries.
4. Create dashboards and visualizations.

*Example 7: Running a SQL query in Databricks SQL*

In [0]:
query = """
SELECT DEST_COUNTRY_NAME, SUM(count) AS total_flights
FROM managed_table
GROUP BY DEST_COUNTRY_NAME
ORDER BY total_flights DESC
LIMIT 10
"""
spark.sql(query).show()

+------------------+-------------+
| DEST_COUNTRY_NAME|total_flights|
+------------------+-------------+
|     United States|       411337|
|            Canada|         8399|
|            Mexico|         7140|
|    United Kingdom|         2025|
|             Japan|         1548|
|           Germany|         1468|
|Dominican Republic|         1353|
|       South Korea|         1048|
|       The Bahamas|          955|
|            France|          935|
+------------------+-------------+



---